<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [31]:
# TODO - Words, words, mere words, no matter from the heart.
from __future__ import print_function

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM


from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import sys
import os

import requests

max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

url = "https://www.gutenberg.org/files/100/100-0.txt"
r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text


#r.encoding = 'utf-8'

In [32]:
len(data)

5740053

In [33]:
# Encode Data as Chars

text = " ".join(data)

chars = list(set(text))

char_int = {c:i for i,c in enumerate(chars)}
int_char = {i:c for i,c in enumerate(chars)}

In [44]:
# Create the Sequence Data

maxlen = 80
step = 10

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 characters long
next_chars = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_chars.append(encoded[i + maxlen])
    
print('sequences:', len(sequences))


sequences: 1148003


In [70]:
sequences_cut = sequences[:100000]
len(sequences_cut)

100000

In [71]:
# Specify x & y

x = np.zeros((len(sequences_cut), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences_cut), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences_cut):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_chars[i]] = 1

In [72]:
x.shape

(100000, 80, 107)

In [73]:
y.shape

(100000, 107)

In [74]:
len(x[67])

80

In [75]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [76]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [67]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=1.0)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [77]:
model.fit(x, y,
          batch_size=128,
          epochs=8,
          callbacks=[print_callback])

Epoch 1/8
 99968/100000 [============================>.] - ETA: 0s - loss: 3.0426
----- Generating text after Epoch: 0
 ---- Generating with seed: " a l l e d   V i n c e n t i o ;   m y   d w e l l i n g   P i s a ; 
 A n d  "
 a l l e d   V i n c e n t i o ;   m y   d w e l l i n g   P i s a ; 
 A n d  erevniedlittthoencey   s-oast fsIunmer.



.  TUGhiun ns, 

  
 


r

 A  v ers   

f
PVEARRooCHN RPMAN.U
 y h a t
 
L
 'R@TI.s

O hEiirnig d  veaynt  oofsliunnt adsp eaetng en's s ,g


    ;

 

xx
Tbulrreyrmeeyimlirs ,s



OB

AC II T E.B
S
F
y  IC



n II CAA
I

H.
 O oDr n-tthyi r ri kelte ept  t jhag iy  teletsl.r


    hR o.A

_F Amwoinasth ey storl
100000/100000 [==============================] - 182s 2ms/sample - loss: 3.0423
Epoch 2/8
 99968/100000 [============================>.] - ETA: 0s - loss: 2.3949
----- Generating text after Epoch: 1
 ---- Generating with seed: "r   V a r r o   a n d   C l a u d i u s . 
 
 V A R R O . 
 C a l l s   m "
    V a r r o   a n d   C l a u 

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN